In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException, InvalidSelectorException, ElementNotVisibleException, WebDriverException
import pickle

In [64]:
category_url = 'https://www.allrecipes.com/recipes/80/main-dish/?page='

def search_entire_category(category_url, save_filename):
    '''writes all recipe links for a given category to pickle list file'''
    MAX_NUM_PAGES = 3 #TODO: find a way to set that automatically
    recipe_urls = []
    
    for page in range(1, MAX_NUM_PAGES):
        recipe_urls.extend(get_recipe_urls(category_url+str(page)))
    
    with open(save_filename, 'wb') as handle:
        pickle.dump(recipe_urls, handle, protocol=pickle.HIGHEST_PROTOCOL)     

In [65]:
def get_recipe_urls(cat_page_url):
    '''returns all recipe links for a given search page within a category'''
    response = requests.get(cat_page_url)
    soup = BeautifulSoup(response.text, "html.parser")
    recipe_urls = []
    
    for elt in set(soup.find_all('a', href=re.compile('^(https://www.allrecipes\.com\/recipe\/)[0-9]*(\/)'))):
        recipe_urls.append(elt['href'])
                   
    return list(set(recipe_urls))

# cat_page_url = 'https://www.allrecipes.com/recipes/80/main-dish/?page=1'
# print(get_recipe_urls(cat_page_url))

In [51]:
def get_recipe_with_reviews_soup(recipe_url):
    '''returns recipe features and recipe reviews'''
    options = webdriver.ChromeOptions()
    options.add_argument('window-size=800x841')
    options.add_argument('headless')
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(weblink)
    driver.implicitly_wait(10)
    
    N_REVIEW_PAGES = 10
    
    for _ in range(N_REVIEW_PAGES):
        try:
            clicker = driver.find_element_by_css_selector('#reviews > div.recipe-reviews__more-container > div.more-button')
            clicker.location_once_scrolled_into_view
            clicker.click()
            time.sleep(1)
        except (TimeoutException, NoSuchElementException, InvalidSelectorException, ElementNotVisibleException, WebDriverException):
            break
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.quit()
    return soup

def get_recipe_reviews(recipe_soup):
    authors = [" ".join(author.text.split()) for author in soup.find_all('h4', attrs={'itemprop': 'author'})]
    ratings = [content['content'] for content in soup.find_all('meta', attrs={'itemprop': 'ratingValue'})][1:] #the first one is the average rating
    dates = ([str(date['content']) for date in soup.find_all('meta', attrs={'itemprop': 'dateCreated'})])
    reviews = list(set(zip(authors, ratings, dates)))
    return reviews
    
def get_recipe_features(recipe_soup):
    title = soup.find('h1', attrs={'id': 'recipe-main-content'}).text
    breadcrumbs = [" ".join(breadcrumb.text.split()) for breadcrumb in soup.find_all('span', attrs={'itemprop': 'name'})]
#   TODO: do we want categories? there's also cuisines but both might be empty depending on the recipe
#     categories = [" ".join(category.text.split()) for category in soup.find_all('span', attrs={'itemprop': 'recipeCategory'})]
    ingredients = [" ".join(ingredient.text.split()) for ingredient in soup.find_all('span', attrs={'itemprop': 'recipeIngredient'})]
    return title, breadcrumbs, ingredients



In [66]:
def generate_db_from_recipe_urls(recipe_urls):
    
    for recipe_url in recipe_urls:
        soup = get_recipe_with_reviews_soup(recipe_url)
        
        recipe_id = re.search('^(https://www.allrecipes\.com\/recipe\/)([0-9]*)(\/)', recipe_url, re.IGNORECASE).group(2)
        reviews = get_recipe_reviews(soup)  
        title, breadcrumbs, ingredients = get_recipe_features(soup)
        
        #TODO: PUT IN DATABASE SOMEHOW
        
    return
    